In [4]:
import requests
from bs4 import BeautifulSoup as BS

In [2]:
URL = 'https://countrymusichalloffame.org/hall-of-fame/members/'

response = requests.get(URL)

In [3]:
response.status_code

200

In [5]:
soup = BS(response.text)
type(soup)

bs4.BeautifulSoup

## 1. Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all inductees? Make use of this to create a list containing just the names of each inductee.

In [9]:
inductee_cards = soup.findAll('div class = vertical-card')
print(type(inductee_cards))
inductee_cards

<class 'bs4.element.ResultSet'>


[]

In [7]:
inductees = [x.get('h3') for x in inductee_cards]

In [10]:
inductees = soup.findAll('h3')
print(type(inductees))
inductees

<class 'bs4.element.ResultSet'>


[<h3>Roy Acuff</h3>,
 <h3>Alabama</h3>,
 <h3>Bill Anderson</h3>,
 <h3>Eddy Arnold</h3>,
 <h3>Chet Atkins</h3>,
 <h3>Gene Autry</h3>,
 <h3>DeFord Bailey</h3>,
 <h3>Bobby Bare</h3>,
 <h3>Eddie Bayers</h3>]

In [11]:
# https://www.geeksforgeeks.org/beautifulsoup-scraping-list-from-html/
# Iterate through all h3 tags
for data in inductees:
    # Get text from each tag
    print(data.text)

Roy Acuff
Alabama
Bill Anderson
Eddy Arnold
Chet Atkins
Gene Autry
DeFord Bailey
Bobby Bare
Eddie Bayers
